In [2]:
print('hello world')

hello world


In [1]:
# import the master_table, get the accession data
import pandas as pd

master_df = pd.read_csv(r"C:\Users\Noah Legall\LegallLab\SHAP-mTB-AMR\resistance_dataset\master_table_resistance.csv")

C:\Users\Noah Legall\AppData\Local\Temp\ipykernel_9336\2610807341.py:4: DtypeWarning: Columns (3,4,7,8,9,12,17,20,22) have mixed types. Specify dtype option on import or set low_memory=False.
  master_df = pd.read_csv(r"C:\Users\Noah Legall\LegallLab\SHAP-mTB-AMR\resistance_dataset\master_table_resistance.csv")


In [2]:
master_df.head()

,index,Isolate,AMIKACIN,AMOXICILLIN,AMOXICILLIN_CLAVULANATE,CAPREOMYCIN,CIPROFLOXACIN,CLARITHROMYCIN,CLOFAZIMINE,CYCLOSERINE,...,OXIFLOXACIN,PARA_AMINOSALICYLIC_ACID,run,run_combined,bioproject,biosample,internal,path,Isolate_original,accessions
0,0,00R0025,R,NaN,NaN,S,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/n/data1/hms/dbmi/farhat/rollingDB/genomic_dat...,00R0025,SAMN05916422
1,1,00R0086,R,NaN,NaN,R,NaN,S,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/n/data1/hms/dbmi/farhat/rollingDB/genomic_dat...,00R0086,SAMN05918543
2,2,00R0178,R,NaN,NaN,R,NaN,R,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/n/data1/hms/dbmi/farhat/rollingDB/genomic_dat...,00R0178,SAMN05918545
3,3,00R0223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/n/data1/hms/dbmi/farhat/rollingDB/genomic_dat...,00R0223,ERS2401270
4,4,00R0312,NaN,NaN,NaN,R,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/n/data1/hms/dbmi/farhat/rollingDB/genomic_dat...,00R0312,ERS2401327


In [4]:
### how many rows are just resistant to rifampicin, and how many rows are susceptible?
rifampicin_susceptible_full = master_df[master_df['RIFAMPICIN'] == 'S']
rifampicin_resistant_full = master_df[master_df['RIFAMPICIN'] == 'R']
print(len(rifampicin_susceptible_full))
print(len(rifampicin_resistant_full))

16136
6480


In [5]:
### return the isolate values for rifampicin resistant and susceptible
rifampicin_susceptible_isolates = rifampicin_susceptible_full['Isolate'].tolist()
rifampicin_resistant_isolates = rifampicin_resistant_full['Isolate'].tolist()

### save the isolate lists to text files
with open(r"C:\Users\Noah Legall\LegallLab\SHAP-mTB-AMR\resistance_dataset\rifampicin_susceptible_isolates.txt", 'w') as f:
    for isolate in rifampicin_susceptible_isolates:
        f.write(f"{isolate}\n")

with open(r"C:\Users\Noah Legall\LegallLab\SHAP-mTB-AMR\resistance_dataset\rifampicin_resistant_isolates.txt", 'w') as f:
    for isolate in rifampicin_resistant_isolates:
        f.write(f"{isolate}\n")

In [8]:
### using the isolate lists, filter out a fasta file that contains the isolate name in the header as regex
from Bio import SeqIO 
fasta_file = r"C:\Users\Noah Legall\LegallLab\SHAP-mTB-AMR\resistance_dataset\rpoBC_20201206.fasta"
fasta_sequences = SeqIO.parse(open(fasta_file),'fasta')
output_fasta = r"C:\Users\Noah Legall\LegallLab\SHAP-mTB-AMR\resistance_dataset\rifampicin_susceptible_rpoBC.fasta"

# Filter and write matching sequences
with open(output_fasta, "w") as out_fh:
    for record in fasta_sequences:
        # Extract the last part of the header (after last '/')
        isolate_id = record.id.split("/")[-1]
        if isolate_id in rifampicin_susceptible_isolates:
            SeqIO.write(record, out_fh, "fasta")



In [9]:
### using the isolate lists, filter out a fasta file that contains the isolate name in the header as regex
from Bio import SeqIO 
fasta_file = r"C:\Users\Noah Legall\LegallLab\SHAP-mTB-AMR\resistance_dataset\rpoBC_20201206.fasta"
fasta_sequences = SeqIO.parse(open(fasta_file),'fasta')
output_fasta = r"C:\Users\Noah Legall\LegallLab\SHAP-mTB-AMR\resistance_dataset\rifampicin_resistant_rpoBC.fasta"

# Filter and write matching sequences
with open(output_fasta, "w") as out_fh:
    for record in fasta_sequences:
        # Extract the last part of the header (after last '/')
        isolate_id = record.id.split("/")[-1]
        if isolate_id in rifampicin_resistant_isolates:
            SeqIO.write(record, out_fh, "fasta")


In [10]:
# Input parameters
input_fasta = r"C:\Users\Noah Legall\LegallLab\SHAP-mTB-AMR\resistance_dataset\rifampicin_resistant_rpoBC.fasta"
output_csv = "sequence_dataset.csv"

# List to store each sequence as a dictionary
data = []

# Process the fasta file
for record in SeqIO.parse(input_fasta, "fasta"):
    # Extract isolate name from the header (last part after "/")
    isolate_id = record.id.split("/")[-1]

    # Convert sequence to uppercase string
    seq_str = str(record.seq).upper()

    # Create dictionary for this sequence
    seq_dict = {f"pos_{i+1}": base for i, base in enumerate(seq_str)}
    seq_dict["isolate_name"] = isolate_id
    seq_dict["resistant"] = True  # Hard-coded as per instruction

    data.append(seq_dict)

# Create DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv(output_csv, index=False)

print(f"Saved dataset with {len(df)} sequences to: {output_csv}")

Saved dataset with 4960 sequences to: sequence_dataset.csv


In [11]:
import os

# Input parameters
input_fasta = r"C:\Users\Noah Legall\LegallLab\SHAP-mTB-AMR\resistance_dataset\rifampicin_susceptible_rpoBC.fasta"
output_csv = "sequence_dataset.csv"

# List to store each sequence as a dictionary
data = []

# Check if CSV exists (to decide whether to write header)
csv_exists = os.path.exists(output_csv)

# Process the fasta file
for record in SeqIO.parse(input_fasta, "fasta"):
    # Extract isolate name from the header (last part after "/")
    isolate_id = record.id.split("/")[-1]

    # Convert sequence to uppercase string
    seq_str = str(record.seq).upper()

    # Create dictionary for this sequence
    seq_dict = {f"pos_{i+1}": base for i, base in enumerate(seq_str)}
    seq_dict["isolate_name"] = isolate_id
    seq_dict["resistant"] = False  # Hard-coded as per instruction

    data.append(seq_dict)

# Create DataFrame
df = pd.DataFrame(data)

# Append to existing CSV (no header if appending)
df.to_csv(output_csv, mode='a', header=not csv_exists, index=False)

print(f"Appended {len(df)} sequences to {output_csv}")

Appended 12400 sequences to sequence_dataset.csv
